#1.Import

In [ ]:
import csv
import cv2
import itertools
import numpy as np
import pandas as pd
import os
import sys
import tempfile

import tensorflow as tf
from enum import Enum
from tqdm import tqdm

from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

#2.Drive mount

In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
image_path = "/content/drive/MyDrive/Wi-Fi/camera_preprocessing/photo_1.jpg"
csv_out_path = "/content/drive/MyDrive/Wi-Fi/camera_preprocessing/output.csv"
images_out_folder = "/content/drive/MyDrive/Wi-Fi/camera_preprocessing"
model_path = "/content/drive/MyDrive/Wi-Fi/camera_preprocessing/movenet_thunder.tflite"

#3.PreProcessing Function

In [ ]:

class BodyPart(Enum):
    """Body part names for each keypoint."""
    NOSE = 0
    LEFT_EYE = 1
    RIGHT_EYE = 2
    LEFT_EAR = 3
    RIGHT_EAR = 4
    LEFT_SHOULDER = 5
    RIGHT_SHOULDER = 6
    LEFT_ELBOW = 7
    RIGHT_ELBOW = 8
    LEFT_WRIST = 9
    RIGHT_WRIST = 10
    LEFT_HIP = 11
    RIGHT_HIP = 12
    LEFT_KNEE = 13
    RIGHT_KNEE = 14
    LEFT_ANKLE = 15
    RIGHT_ANKLE = 16

def load_movenet_model(model_path):
    """Loads the MoveNet model from the given .tflite file.

    Args:
        model_path: Path to the MoveNet .tflite file.

    Returns:
        Loaded MoveNet interpreter.
    """
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    return interpreter

def detect_landmarks(image, interpreter):
    """Detects landmarks in the given image using the MoveNet model.

    Args:
        image: Input image in RGB format.
        interpreter: MoveNet interpreter.

    Returns:
        Detected landmarks.
    """
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    input_shape = input_details[0]['shape']
    input_tensor = np.expand_dims(image, axis=0)
    input_tensor = tf.image.resize(input_tensor, (input_shape[1], input_shape[2]))
    input_tensor = tf.cast(input_tensor, dtype=input_details[0]['dtype'])

    interpreter.set_tensor(input_details[0]['index'], input_tensor)
    interpreter.invoke()

    output_tensor = interpreter.get_tensor(output_details[0]['index'])
    landmarks = output_tensor.squeeze()
    print(landmarks)
    return landmarks

def preprocess_single_image(image_path, csv_out_path, images_out_folder, model_path, detection_threshold=0.1):
    """Preprocesses a single image and saves the detected landmarks in a CSV file.

    Args:
        image_path: Path to the input image.
        csv_out_path: Path to write the CSV file containing the detected landmark
            coordinates.
        images_out_folder: Path to write the image overlay with detected
            landmarks.
        model_path: Path to the MoveNet .tflite file.
        detection_threshold: Only keep the image if all landmark confidence scores
            are above this threshold.
    """
    # Load the image
    try:
        image = tf.io.read_file(image_path)
        image = tf.io.decode_jpeg(image)
    except:
        raise ValueError(f"Invalid image: {image_path}")

    image_height, image_width, channel = image.shape

    # Skip images that aren't RGB because MoveNet requires RGB images
    if channel != 3:
        raise ValueError(f"Image isn't in RGB format: {image_path}")


    # Load the MoveNet model
    interpreter = load_movenet_model(model_path)

    # Detect landmarks using MoveNet
    landmarks = detect_landmarks(image, interpreter)

    # Write the landmark coordinates to the CSV file
    with open(csv_out_path, "w") as csv_out_file:
        csv_out_writer = csv.writer(csv_out_file, delimiter=",", quoting=csv.QUOTE_MINIMAL)
        coordinates = landmarks.flatten().astype(str).tolist()
        csv_out_writer.writerow([os.path.basename(image_path)] + coordinates)


In [ ]:
preprocess_single_image(image_path, csv_out_path, images_out_folder, model_path)
# 각각의 좌표값

[[0.02797922 0.49135107 0.01280772]
 [0.01546238 0.5675123  0.00785297]
 [0.01129412 0.46207592 0.01420071]
 [0.00710539 0.67704993 0.0197892 ]
 [0.00119213 0.47105625 0.01207166]
 [0.01198644 0.7015546  0.02657878]
 [0.00808639 0.50048876 0.02618898]
 [0.48540503 0.8064845  0.03578768]
 [0.5131134  0.34670332 0.01114855]
 [0.6355806  0.8317252  0.01557579]
 [0.6497566  0.35922277 0.00938828]
 [0.6646994  0.6911843  0.06165674]
 [0.6780077  0.412466   0.04144444]
 [0.891289   0.63395095 0.02103065]
 [0.9025091  0.49753556 0.01624199]
 [0.9293704  0.7236801  0.01250068]
 [0.92470306 0.65199095 0.02768951]]
